In [5]:
from fazbot.db.fazbot import FazBotDatabase

db = FazBotDatabase(
    user='test',
    password='test',
    database='test_fazbot',
    retries=3
)

await db.create_all_tables()

await db.banned_user_repository.create_table()
tables = await db._db.fetch(
    'SHOW TABLES;'
)
tables

[{'Tables_in_test_fazbot': 'banned_user'},
 {'Tables_in_test_fazbot': 'test_table'}]